In [1]:
sc

In [3]:
#Start H2O
from pysparkling import *
sc
hc= H2OContext(sc).start()

Py4JJavaError: An error occurred while calling o29.loadClass.
: java.lang.ClassNotFoundException: org.apache.spark.h2o.H2OContext
	at java.net.URLClassLoader$1.run(URLClassLoader.java:366)
	at java.net.URLClassLoader$1.run(URLClassLoader.java:355)
	at java.security.AccessController.doPrivileged(Native Method)
	at java.net.URLClassLoader.findClass(URLClassLoader.java:354)
	at java.lang.ClassLoader.loadClass(ClassLoader.java:425)
	at java.lang.ClassLoader.loadClass(ClassLoader.java:358)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:57)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:606)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:231)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:379)
	at py4j.Gateway.invoke(Gateway.java:259)
	at py4j.commands.AbstractCommand.invokeMet

In [ ]:
hc

In [ ]:
import h2o

In [ ]:
# Parse Chicago Crime dataset 
column_type = ['Numeric','String','String','Enum','Enum','Enum','Enum','Enum','Enum','Enum','Numeric','Numeric','Numeric','Numeric','Enum','Numeric','Numeric','Numeric','Enum','Numeric','Numeric','Enum']
f_crimes = h2o.import_file(path ="/Users/nidhimehta/sparkling-water/examples/smalldata/chicagoCrimes10k.csv",col_types =column_type)

print(f_crimes.shape)
f_crimes.summary()

In [ ]:
f_crimes["IUCR"].table()

In [ ]:
f_crimes["Arrest"].table()

In [ ]:
col_names = map(lambda s: s.replace(' ', '_'), f_crimes.col_names)
f_crimes.set_names(col_names)

h2o.set_timezone("Etc/UTC")

## Refine date column
def refine_date_col(data, col, pattern):
    data[col]         = data[col].as_date(pattern)
    data["Day"]       = data[col].day()
    data["Month"]     = data[col].month()    # Since H2O indexes from 0
    data["Year"]      = data[col].year()
    data["WeekNum"]   = data[col].week()
    data["WeekDay"]   = data[col].dayOfWeek()
    data["HourOfDay"] = data[col].hour()
    
    #data.describe()  # HACK: Force evaluation before ifelse and cut. See PUBDEV-1425.
    
    # Create weekend and season cols
    data["Weekend"] = (data["WeekDay"] == "Sun" or data["WeekDay"] == "Sat").ifelse(1, 0)[0]
    data["Season"] = data["Month"].cut([0, 2, 5, 7, 10, 12], ["Winter", "Spring", "Summer", "Autumn", "Winter"])
    
refine_date_col(f_crimes, "Date", "%m/%d/%Y %I:%M:%S %p")
f_crimes = f_crimes.drop("Date")
#f_crimes["Season"].nlevels()

In [ ]:
#Parse census data
f_census = h2o.import_file("/Users/nidhimehta/Desktop/Chicago/chicagoCensus.csv",header=1)
#f_census = h2o.import_file("/Users/nidhimehta/sparkling-water/examples/smalldata/chicagoCensus.csv",header=1)

## Update column names in the table
col_names = map(lambda s: s.strip().replace(' ', '_'), f_census.col_names)
f_census.set_names(col_names)
f_census = f_census[1:78,:]
f_census.summary()

In [ ]:
f_weather = h2o.import_file("/Users/nidhimehta/Desktop/Chicago/chicagoAllWeather.csv")
f_weather = f_weather[1:]
f_weather.summary()

In [ ]:
f_weather[f_weather["meanTemp"].isna()]

In [ ]:
hc.as_h2o_frame?

In [ ]:
#Copy data frames to Spark from H2O
df_weather = hc.as_data_frame(f_weather)
df_census = hc.as_data_frame(f_census)
df_crimes = hc.as_data_frame(f_crimes)

In [ ]:
df_weather.show(2)

In [ ]:
## Register DataFrames as tables in SQL context
sqlContext.registerDataFrameAsTable(df_weather, "chicagoWeather")
sqlContext.registerDataFrameAsTable(df_census, "chicagoCensus")
sqlContext.registerDataFrameAsTable(df_crimes, "chicagoCrime")


crimeWithWeather = sqlContext.sql("""SELECT
a.Year, a.Month, a.Day, a.WeekNum, a.HourOfDay, a.Weekend, a.Season, a.WeekDay,
a.IUCR, a.Primary_Type, a.Location_Description, a.Community_Area, a.District,
a.Arrest, a.Domestic, a.Beat, a.Ward, a.FBI_Code,
b.minTemp, b.maxTemp, b.meanTemp,
c.PERCENT_AGED_UNDER_18_OR_OVER_64, c.PER_CAPITA_INCOME, c.HARDSHIP_INDEX,
c.PERCENT_OF_HOUSING_CROWDED, c.PERCENT_HOUSEHOLDS_BELOW_POVERTY,
c.PERCENT_AGED_16__UNEMPLOYED, c.PERCENT_AGED_25__WITHOUT_HIGH_SCHOOL_DIPLOMA
FROM chicagoCrime a
JOIN chicagoWeather b
ON a.Year = b.year AND a.Month = b.month AND a.Day = b.day
JOIN chicagoCensus c
ON a.Community_Area = c.Community_Area_Number""")

In [ ]:
crimeWithWeather.show(2)

In [ ]:
#Copy table from Spark to H2O
crimeWithWeatherHF = hc.as_h2o_frame(crimeWithWeather,framename="crimeWithWeather")


In [ ]:
crimeWithWeatherHF.summary()

In [ ]:
#Assign Column tyoes
crimeWithWeatherHF["Season"]= crimeWithWeatherHF["Season"].asfactor()
crimeWithWeatherHF["WeekDay"]= crimeWithWeatherHF["WeekDay"].asfactor()
crimeWithWeatherHF["IUCR"]= crimeWithWeatherHF["IUCR"].asfactor()
crimeWithWeatherHF["Primary_Type"]= crimeWithWeatherHF["Primary_Type"].asfactor()
crimeWithWeatherHF["Location_Description"]= crimeWithWeatherHF["Location_Description"].asfactor()
crimeWithWeatherHF["Arrest"]= crimeWithWeatherHF["Arrest"].asfactor()
crimeWithWeatherHF["Domestic"]= crimeWithWeatherHF["Domestic"].asfactor()
crimeWithWeatherHF["FBI_Code"]= crimeWithWeatherHF["FBI_Code"].asfactor()
crimeWithWeatherHF["Season"]= crimeWithWeatherHF["Season"].asfactor()


In [ ]:
crimeWithWeatherHF.summary()

In [ ]:
H2OGradientBoostingEstimator?

In [ ]:
# Split final data table
ratios = [0.8]
frs = crimeWithWeatherHF.split_frame(ratios)
train = frs[0]
test = frs[1]

In [ ]:

from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.estimators.deeplearning import H2ODeepLearningEstimator

# Simple GBM - Predict Arrest
crimes_names_x = crimeWithWeatherHF.names
print(crimes_names_x)
crimes_names_x.remove("Arrest")


In [ ]:
#Simple GBM model - Predict Arrest
model_gbm = H2OGradientBoostingEstimator(ntrees         =50,
                                        max_depth      =6,
                                        learn_rate     =0.1, 
                                        #nfolds         =5,
                                        distribution   ="bernoulli")

model_gbm.train(x               =crimes_names_x,
               y               ="Arrest",
               training_frame  =train,
               validation_frame=test)

# Simple Deep Learning - Predict Arrest
model_dl = H2ODeepLearningEstimator(variable_importances=True,
                                   loss                ="Automatic")

model_dl.train(x                =crimes_names_x,
              y                ="Arrest",
              training_frame  =train,
              validation_frame=test)

In [ ]:
print(model_gbm.confusion_matrix(train = True))
print(model_gbm.confusion_matrix(valid = True))

In [ ]:
model_gbm.plot(metric="AUC")

In [ ]:
#Print variable importance
model_gbm.varimp()

In [ ]:
model_dl

In [ ]:
# Predict on the test set using the cross-validated gbm model

predictions = model_gbm.predict(test)
predictions.show()

In [ ]:
# Look at performance of test set (if it includes true lables)
test_performance = model_gbm.model_performance(test)
test_performance

In [ ]:
#Plots

In [ ]:
sqlContext.registerDataFrameAsTable(df_crimes, "df_crimes")
allCrimes = sqlContext.sql("""SELECT Primary_Type, count(*) as all_count FROM df_crimes GROUP BY Primary_Type""")
crimesWithArrest = sqlContext.sql("SELECT Primary_Type, count(*) as crime_count FROM chicagoCrime WHERE Arrest = 'true' GROUP BY Primary_Type")

In [ ]:
sqlContext.registerDataFrameAsTable(crimesWithArrest, "crimesWithArrest")
sqlContext.registerDataFrameAsTable(allCrimes, "allCrimes")
crime_type = sqlContext.sql("Select a.Primary_Type as Crime_Type, a.crime_count, b.all_count \
FROM crimesWithArrest a \
JOIN allCrimes b \
ON a.Primary_Type = b.Primary_Type ")

In [ ]:
crime_type.show(12)

In [ ]:
#Copy table from Spark to H2O
crime_typeHF = hc.as_h2o_frame(crime_type,framename="crime_type")

In [ ]:
# Create new columns Arrest_rate and Crime_propotion 
crime_typeHF["Arrest_rate"] = crime_typeHF["crime_count"]/crime_typeHF["all_count"]
crime_typeHF["Crime_proportion"] = crime_typeHF["all_count"]/crime_typeHF["all_count"].sum()
# h2o.assign(crime_typeHF,crime_type)
crime_typeHF["Crime_Type"] = crime_typeHF["Crime_Type"].asfactor()
crime_typeHF.frame_id = "Crime_type"

In [ ]:
crime_typeHF

In [ ]:
h2o.cluster_info()
#dir(h2o)

In [ ]:
# Plot the below in Flow 
plot (g) -> g(
  g.rect(
    g.position "Crime_Type", "Arrest_rate"
    g.fillColor g.value 'blue'
    g.fillOpacity g.value 0.75
  )
  g.rect(
    g.position "Crime_Type", "Crime_proportion"
    g.fillColor g.value 'red'
    g.fillOpacity g.value 0.65
  )
  g.from inspect "data", getFrame "Crime_type"
)

In [ ]:
hc.stop()

In [ ]:
#Create new H2OFrame of crime observations
#examples = {
            "Date":                 ["02/08/2015 11:43:58 PM", "02/08/2015 11:00:39 PM"],
            "IUCR":                 [1811, 1150],
            "Primary.Type":         ["NARCOTICS", "DECEPTIVE PRACTICE"],
            "Location.Description": ["STREET", "RESIDENCE"],
            "Domestic":             ["false", "false"],
            "Beat":                 [422, 923],
            "District":             [4, 9],
            "Ward":                 [7, 14],
            "Community.Area":       [46, 63],
            "FBI.Code":             [18, 11]
            }

crime_examples = h2o.H2OFrame(python_obj = examples)



In [ ]:
# Refine date column and merge with census data
#refine_date_col(crime_examples, "Date", "%m/%d/%Y %I:%M:%S %p")
#crime_examples.drop("Date")
#f_census.set_name(0,"Community.Area")
#crime_examples.merge(f_census, allLeft=True, allRite=False)
#crime_examples.describe()

In [ ]:
train_auc_gbm = gbm.model_performance(train).auc()


In [ ]:
f_crimes["Arrest"].table()